In [1]:

import face_recognition
import matplotlib.pyplot as plt
import cv2
def compare(input_img ,webcam_img): 
        img1 = face_recognition.load_image_file(input_img) 
        img2 = face_recognition.load_image_file(webcam_img) 

        en1 = face_recognition.face_encodings(img1)[0]
        # print(en1)
        en2 = face_recognition.face_encodings(img2)[0]

        a=face_recognition.compare_faces([en1] , en2)
        return a[0]


In [2]:


cap = cv2.VideoCapture(0)

while True:
    succes , img = cap.read()
    cv2.imshow("frame", img)

    key =cv2.waitKey(1)
    if key == ord('p'):  
        cv2.imwrite("webcam_photo.png", img)
        cap.release()
        cv2.destroyAllWindows()
        try:
            a =compare("myimage.jpg","webcam_photo.png")
            # a =compare("myimage.jpg",img)
    
            if(a==False):
                print("Faces Does NOT match")
                break
            else:
                print("Faces Match")
                break
        except :
            print("no faces detected")
            break
    
    # cv2.imshow("frame", img)
# cap.release()
# cv2.destroyAllWindows()    


Faces Match


In [3]:
import face_recognition
import cv2

def compare(input_img, webcam_img):
    img1 = face_recognition.load_image_file(input_img)
    en1 = face_recognition.face_encodings(img1)[0]

    img2 = cv2.cvtColor(webcam_img, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(img2)

    if len(encodings) > 0:
        en2 = encodings[0]
        a = face_recognition.compare_faces([en1], en2)
        return a[0]
    else:
        return None

reference_image = "myimage.jpg"
cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break

    match = compare(reference_image, frame)

    if match is None:
        text = "No face detected"
    elif match:
        text = "Faces Match"
        break
    else:
        text = "Faces Do NOT Match"

    cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("frame", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
import face_recognition
import cv2

def compare(input_img, webcam_img):
    img1 = face_recognition.load_image_file(input_img)
    en1 = face_recognition.face_encodings(img1)[0]

    # Convert the frame to RGB (since face_recognition uses RGB images)
    img2 = cv2.cvtColor(webcam_img, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(img2)

    if len(encodings) > 0:
        en2 = encodings[0]
        a = face_recognition.compare_faces([en1], en2)
        return a[0]
    else:
        return None

reference_image = "myimage.jpg"
cap = cv2.VideoCapture(0)

# Reduce frame size for faster processing
frame_reduction_factor = 2  # Change this factor based on your system's capability

while True:
    success, frame = cap.read()
    if not success:
        break

    # Resize the frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=1.0 / frame_reduction_factor, fy=1.0 / frame_reduction_factor)

    # Perform the face comparison
    match = compare(reference_image, small_frame)

    # Upscale the result text position to match the original frame size
    text_position = (50 * frame_reduction_factor, 50 * frame_reduction_factor)

    if match is None:
        text = "No face detected"
    elif match:
        text = "Faces Match"
        print("success")
        break
    else:
        text = "Faces Do NOT Match"

    cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("frame", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


success


In [7]:
import face_recognition
import cv2
import threading

def compare(input_img, webcam_img):
    img1 = face_recognition.load_image_file(input_img)
    en1 = face_recognition.face_encodings(img1)[0]

    # Convert the frame to RGB (since face_recognition uses RGB images)
    img2 = cv2.cvtColor(webcam_img, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(img2)

    if len(encodings) > 0:
        en2 = encodings[0]
        a = face_recognition.compare_faces([en1], en2)
        return a[0]
    else:
        return None

reference_image = "myimage.jpg"
cap = cv2.VideoCapture(0)

# Reduce frame size for faster processing
frame_reduction_factor = 2  # Change this factor based on your system's capability
skip_frames = 5  # Process every 5th frame

# For multi-threaded frame capture
frame = None
lock = threading.Lock()
running = True

def capture_frames():
    global frame
    while running:
        success, img = cap.read()
        if success:
            with lock:
                frame = img

# Start frame capture thread
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

frame_count = 0

while True:
    with lock:
        if frame is None:
            continue
        current_frame = frame.copy()

    if frame_count % skip_frames == 0:
        # Resize the frame for faster processing
        small_frame = cv2.resize(current_frame, (0, 0), fx=1.0 / frame_reduction_factor, fy=1.0 / frame_reduction_factor)

        # Perform the face comparison
        match = compare(reference_image, small_frame)

        # Upscale the result text position to match the original frame size
        text_position = (50 * frame_reduction_factor, 50 * frame_reduction_factor)

        if match is None:
            text = "No face detected"
        elif match:
            text = "Faces Match"
            print("success")
            break
        else:
            text = "Faces Do NOT Match"

        cv2.putText(current_frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow("frame", current_frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

    frame_count += 1

running = False
capture_thread.join()
cap.release()
cv2.destroyAllWindows()


success
